In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
#Import CSV from WeatherPy to create a DataFrame
df = pd.read_csv (r'../WeatherPy/output_data/cities.csv')
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,half moon bay,37.46,-122.43,15.00,93,90,4.60,US,1596628534
1,1,dingle,11.00,122.67,25.55,87,100,1.77,PH,1596628534
2,2,marondera,-18.19,31.55,24.21,20,0,2.36,ZW,1596628535
3,3,dossor,47.53,52.98,28.34,30,85,4.76,KZ,1596628535
4,4,busselton,-33.65,115.33,12.22,86,34,0.89,AU,1596628535
...,...,...,...,...,...,...,...,...,...,...
550,550,nishihara,26.18,127.76,29.00,83,75,4.10,JP,1596628637
551,551,omboue,-1.57,9.26,24.76,72,97,3.58,GA,1596628637
552,552,bathsheba,13.22,-59.52,28.00,74,40,7.20,BB,1596628637
553,553,yugo-kamskiy,57.71,55.59,24.00,69,20,5.00,RU,1596628637


# Humidity Heatmap

In [3]:
# Store latitude and longitude
cities = df[["Lat", "Lng"]]

# Store cities
humidity = df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#Create new DataFrame fitting weather criteria
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = df.loc[(df["Max Temp"] > 21) & (df["Max Temp"] < 26) & (df["Cloudiness"] == 0) & (df["Wind Speed"] < 10)
                  & (df["Humidity"] <= 50), :]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,marondera,-18.19,31.55,24.21,20,0,2.36,ZW,1596628535
188,188,rehoboth,-23.32,17.08,25.14,15,0,8.09,NaN,1596628570
316,316,diamantino,-14.41,-56.45,25.18,33,0,3.68,BR,1596628591
430,430,burevestnik,56.14,43.79,25.00,44,0,3.00,RU,1596628610


In [5]:
# Store latitude and longitude in coordinates
hotel_df = ideal_df[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"]= ""
hotel_df

C:\Users\Mabel\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Lat,Lng,Hotel Name
2,marondera,ZW,-18.19,31.55,
188,rehoboth,NaN,-23.32,17.08,
316,diamantino,BR,-14.41,-56.45,
430,burevestnik,RU,56.14,43.79,


In [10]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}
# Iterate through
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    # convert to json
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

C:\Users\Mabel\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Lng,Hotel Name
2,marondera,ZW,-18.19,31.55,HopeFay Conference Centre
188,rehoboth,NaN,-23.32,17.08,Noble Luminous Accommodation
316,diamantino,BR,-14.41,-56.45,Hotel Delta
430,burevestnik,RU,56.14,43.79,Oreshkino


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))